In [1]:
# installing talib
!tar -xzf ta-lib-0.4.0-src.tar.gz
cd ta-lib/

!./configure --prefix=/usr

!make

!sudo make install

cd ../

!pip install TA-Lib

SyntaxError: invalid syntax (1867775520.py, line 3)

In [13]:
!pip install mlflow

In [14]:
!pip install dtw-python

In [15]:
!pip install kneed

In [16]:
!pip install snowflake-connector-python

In [17]:
pip install llvmlite --ignore-installed

  Using cached llvmlite-0.38.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device

Note: you may need to restart the kernel to use updated packages.


In [18]:
!pip install --pre pycaret

In [19]:
!pip install pyarrow==6.0.0

In [2]:
from preprocessing_utils import preprocess1, get_all_indices
from formatting_utils import load_all_format_components_gcs, data_formatter
from modeling_utils import get_split_dates, pycaret_automl, prepare_data, split_df, add_target, get_cross_corr_matrix
import yaml
import snowflake.connector
import numpy as np
from datetime import datetime
from data_loading_utils import load_crypto, crypto_filter_times

# import time
# from kneed import DataGenerator, KneeLocator
# from sklearn.ensemble import IsolationForest
# from scipy.stats import rankdata
# import matplotlib.pyplot as plt
# import os
# from snowflake.connector.pandas_tools import write_pandas
# import snowflake.connector
# import pandas as pd
# import yaml
# from datetime import datetime
# from getpass import getpass
# import pickle
# import numpy as np
# from tqdm import tqdm
# from sklearn.cluster import KMeans
# from tqdm import tqdm
# from itertools import combinations
# import warnings
# from pycaret.classification import *
import traceback
# import gcsfs

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)


#===========================================================
#====================FEATURE ENGINEERING====================
#===========================================================

# def clusterize(df, cluster_num=2):
#     def find_cluster_num(data, max_clusters=10):
#         sse = {}
#         for k in range(1, max_clusters + 1):
#             kmeans = KMeans(n_clusters=k)
#             kmeans.fit(data)
#             sse[k] = kmeans.inertia_
#         kn = KneeLocator(x=list(sse.keys()), 
#                   y=list(sse.values()), 
#                   curve='convex', 
#                   direction='decreasing')
#         return kn.knee 
#     df_out= df.copy()
#     clust_model_dict = {}
#     order_dict = {}
#     for col in tqdm(df.columns):
#         col_output = df[col]
#         col_output = col_output.replace([np.inf, -np.inf], np.nan)
#         with warnings.catch_warnings():
#             warnings.simplefilter("ignore")
#             if cluster_num is None:
#                 cluster_num_tmp = find_cluster_num(col_output.dropna().values.reshape(-1, 1))
#             else:
#                 cluster_num_tmp = cluster_num
#             kmeans = KMeans(n_clusters=cluster_num_tmp).fit(col_output.dropna().values.reshape(-1, 1))
#             clust_model_dict[col] = kmeans
#             order_map = dict(zip(range(cluster_num_tmp), np.squeeze(rankdata(kmeans.cluster_centers_)).tolist()))
#             col_output[~col_output.isna()] = list(map(lambda x: order_map[x], kmeans.labels_))
#             order_dict[col] = order_map
#         df_out[col] = col_output
#     return df_out, clust_model_dict, order_dict


#===============================================
#====================PYCARET====================
#===============================================

# def pycaret_automl(crypto_train, crypto_test, coin, return_period, 
#                    plot=True, download=False, save_folder="/content/drive/MyDrive/crypto_models",
#                    pycaret_setup_args=None,
#                    pycaret_compare_args=None):
#     # Setting up environment
#     # CRITICAL: Out-of-time validation scheme
#     if pycaret_setup_args is None:
#         exp_default = setup(data=crypto_train, test_data=crypto_test, target="target", log_experiment=True, 
#                             experiment_name="{}_{}".format(coin, return_period),)
#     else:

#         exp_default = setup(data=crypto_train, test_data=crypto_test, target="target", log_experiment=True, 
#                     experiment_name="{}_{}".format(coin, return_period), **pycaret_setup_args)
    
#     if save_folder is not None:
#         if os.path.exists(save_folder):
#             pass
#         else:
#             os.makedirs(save_folder)

#     start_time = time.time()
#     if pycaret_compare_args is None:
#         best_models = compare_models(n_select=5, sort="AUC", exclude=["gbc"])
#     else:
#         if "include" in pycaret_compare_args:
#             best_models = compare_models(n_select=5, sort="AUC", **pycaret_compare_args)
#         else:
#             best_models = compare_models(n_select=5, sort="AUC", exclude=["gbc"], **pycaret_compare_args)
#     print("Completed in {} seconds".format(time.time() - start_time))

#     if download:
#         files.download(mld_name + ".pkl")
  
#     best_model = best_models[0]# Selecting best model

#     if plot:
#         print("========================================================================================================================================")
#         print("===============================================================BEST MODEL===============================================================")
#         print("========================================================================================================================================")
#         plot_model(best_model)
#         plot_model(best_model, plot="confusion_matrix")

#         print("========================================================================================================================================")
#         print("===============================================================BEST BLEND===============================================================")
#         print("========================================================================================================================================")
#         plot_model(best_models_blend)
#         plot_model(best_models_blend, plot="confusion_matrix")
      
#     # Finalizing best model(s)
#     best_model_finalized = finalize_model(best_model)
  
#     if save_folder is not None:
#         if save_folder.endswith("/"):
#             get_logs().to_csv(save_folder + "{}_{}_log.csv".format(coin, return_period))
#             save_model(best_model_finalized, save_folder + "best_model_finalized")
#         else:
#             get_logs().to_csv(save_folder + "/{}_{}_log.csv".format(coin, return_period))
#             save_model(best_model_finalized, save_folder + "/best_model_finalized")
#     else:
#         save_model(best_model_finalized, "best_model_finalized")
#         get_logs().to_csv("{}_{}_log.csv".format(coin, return_period))
  
#     return best_models, best_model_finalized, get_logs()



#=========================================================
#====================CROSS-CORRELATION====================
#=========================================================

# def cross_corr(crypto_wide, coin1, coin2, lag):
#     if lag > 0:
#         return(np.corrcoef(crypto_wide.dropna(subset=[coin1, coin2])[coin1].iloc[lag:], crypto_wide.dropna(subset=[coin1, coin2])[coin2].shift(lag).dropna())[0, 1])
#     elif lag < 0:
#         return(np.corrcoef(crypto_wide.dropna(subset=[coin1, coin2])[coin1].iloc[:lag], crypto_wide.dropna(subset=[coin1, coin2])[coin2].shift(lag).dropna())[0, 1])
#     elif lag == 0:
#         return(np.corrcoef(crypto_wide.dropna(subset=[coin1, coin2])[coin1], crypto_wide.dropna(subset=[coin1, coin2])[coin2])[0, 1])


# def cross_corr_range(crypto_wide, coin1, coin2, lag_range, plot=True):
#     result = pd.Series(lag_range).apply(lambda x: cross_corr(crypto_wide, coin1, coin2, int(round(x))))
#     result.index = lag_range
#     if plot:
#         plt.plot(result)
#         plt.show()
#     else:
#         pass
#     return result


# def get_cross_corr_matrix(crypto_wide, price_cols, num_est = 10):
#     cross_corr_mat = pd.DataFrame([], columns=price_cols, index=price_cols)
#     print("Getting cross-correlations")
#     print("\t", end="")
#     for pair in tqdm(list(combinations(price_cols, 2))):
#         cross_corr_values = cross_corr_range(crypto_wide, pair[0], pair[1], np.linspace(-2880, 2880, num=num_est), plot=False)
#         greatest_result = cross_corr_values[cross_corr_values == cross_corr_values.max()]
#         greatest_lag = greatest_result.index[0]
#         greatest_corr = greatest_result.values[0]
#         if greatest_lag < 0:
#             cross_corr_mat.loc[pair[0], pair[1]] = greatest_corr
#         elif greatest_lag > 0:
#             cross_corr_mat.loc[pair[1], pair[0]] = greatest_corr
#         else:
#             pass
#     return cross_corr_mat.astype(float)


# def get_split_dates(crypto_wide, num_weeks_train, num_weeks_test, num_weeks_holdout):
#     # Defining our periods of interest
#     holdout_start = crypto_wide.index.max() - pd.Timedelta(weeks=num_weeks_holdout)
#     test_start = holdout_start - pd.Timedelta(weeks=num_weeks_test)
#     train_start = test_start - pd.Timedelta(weeks=num_weeks_train)
#     print("Number of training samples: {}".format(((crypto_wide.index < test_start) & (crypto_wide.index >= train_start)).sum()))
#     print("Number of test samples: {}".format(((crypto_wide.index >= test_start) & (crypto_wide.index < holdout_start)).sum()))
#     print("Number of holdout samples: {}".format(((crypto_wide.index >= holdout_start)).sum()))
#     return train_start, test_start, holdout_start


# def prepare_data(crypto_wide, coin_of_interest, columns_of_interest, train_start, test_start, holdout_start, cluster_num=None, 
#                   cross_correlation_matrix=None, cross_corr_thresh=0.50, outlier_column=False):
#     all_coins = [coin_of_interest]

#     # Yielding our testing/training/holdout dataframes
#     crypto_train = crypto_wide.loc[train_start:test_start]
#     crypto_test = crypto_wide.loc[test_start:holdout_start]
#     crypto_holdout = crypto_wide.loc[holdout_start:]
#     crypto_test_holdout = pd.concat([crypto_test, crypto_holdout])

#     # Getting clusters for each coin return
#     print("Fitting clusters on training data")
#     print("\t", end="")
#     cluster_cols = [col for col in crypto_train if "{}_pct_change".format(coin_of_interest) in col]
#     crypto_return_clusters_train, cluster_model_dict, order_dict = clusterize(crypto_wide.loc[train_start:test_start, cluster_cols], cluster_num=cluster_num)
# #     crypto_return_clusters_train = crypto_return_clusters_train.loc[train_start:]
#     crypto_return_clusters_test_holdout = crypto_test_holdout[cluster_cols].copy()

#     print("Predicting clusters on test and holdout data")
#     print("\t", end="")
#     for col in tqdm(cluster_cols):
#         kmean_labels = cluster_model_dict[col].predict(crypto_return_clusters_test_holdout[col].values.reshape(-1, 1))
#         crypto_return_clusters_test_holdout[col] = list(map(lambda x: order_dict[col][x], kmean_labels))
    
#     crypto_return_clusters_train = crypto_return_clusters_train.astype(int).astype(str)
#     crypto_return_clusters_test_holdout = crypto_return_clusters_test_holdout.astype(int).astype(str)
#     crypto_train = crypto_train.join(crypto_return_clusters_train, lsuffix="", rsuffix="_clust")
#     crypto_test_holdout = crypto_test_holdout.join(crypto_return_clusters_test_holdout, lsuffix="", rsuffix="_clust")
#     columns_of_interest += [col for col in crypto_train if "clust" in col]
  
#     # Adding coin data for potentially causal coins
#     if cross_correlation_matrix is not None:
#         print("Getting additional potentially influential coins")
#         print("\t", end="")
#         additional_coins = (cross_correlation_matrix.abs() > cross_corr_thresh).query(coin_of_interest).index.tolist()
#         all_coins += additional_coins
#         additional_cols = []
#         for coin in tqdm(additional_coins):
#             additional_cols += [col for col in crypto_train if coin in col]
#         columns_of_interest += additional_cols
#         columns_of_interest = list(set(columns_of_interest))
#     else:
#         pass
  
#     # Getting coin-based outliers
#     if outlier_column:
#         print("Getting outliers")
#         print("\t", end="")
#         outlier_dict = {}
#         for coin in tqdm(all_coins):
#             # Getting coin-related columns to create outlier computation
#             coin_cols = [col for col in crypto_wide if coin in col]

#             # training the model
#             clf = IsolationForest(max_samples=100, random_state=1)
#             clf.fit(crypto_train[coin_cols].replace([np.inf, -np.inf], np.nan).dropna().values)
#             outlier_dict[coin] = (clf, coin_cols)
#             train_outliers = clf.predict(crypto_train[coin_cols].replace([np.inf, -np.inf], np.nan).dropna().values)
#             outlier_col = "{}_OUTLIER".format(coin)
#             columns_of_interest += [outlier_col]
#             crypto_train[outlier_col] = np.nan
#             crypto_train.loc[~crypto_train[coin_cols].replace([np.inf, -np.inf], np.nan).isna().any(axis=1), outlier_col] = train_outliers.astype(int).astype(str)

#             # predicting model on test/holdout partitions
#             test_outliers = clf.predict(crypto_test_holdout[coin_cols].replace([np.inf, -np.inf], np.nan).dropna().values)
#             crypto_test_holdout[outlier_col] = np.nan
#             crypto_test_holdout.loc[~crypto_test_holdout[coin_cols].replace([np.inf, -np.inf], np.nan).isna().any(axis=1), outlier_col] = test_outliers.astype(int).astype(str)
  
#     # Selecting only our columns of interest
#     crypto_train = crypto_train[columns_of_interest]
#     crypto_test_holdout = crypto_test_holdout[columns_of_interest]
#     return crypto_train, crypto_test_holdout, crypto_return_clusters_train, crypto_return_clusters_test_holdout, \
#             cluster_model_dict, outlier_dict


# def add_target(coin_of_interest, crypto_train, crypto_test_holdout, return_period, test_start):
#     # Joining on clusters of our coin returns
#     target = "{}_pct_change_{}".format(coin_of_interest, return_period)
#     combined_df = pd.concat([crypto_train, crypto_test_holdout])
#     combined_cluster_df = pd.concat([crypto_return_clusters_train, crypto_return_clusters_test_holdout])
#     combined_df = combined_df[~combined_df.index.duplicated(keep='first')]
#     combined_cluster_df = combined_cluster_df[~combined_cluster_df.index.duplicated(keep='first')]
#     combined_df["target"] = combined_cluster_df[target].shift(-return_period)
#     crypto_train, crypto_test_holdout = combined_df.loc[:test_start], combined_df.loc[test_start:]
#     cluster_desc = crypto_train.groupby(crypto_return_clusters_train[target])[[target]].describe()
#     print("Unique values of target: {}".format(crypto_train["target"].nunique()))
#     return crypto_train, crypto_test_holdout, cluster_desc


# def split_df(crypto_train, crypto_test_holdout, holdout_start):
#     # Further partitioning our combined test/holdout dataframe
#     crypto_test, crypto_holdout = crypto_test_holdout.loc[:holdout_start].dropna(subset=["target"]), crypto_test_holdout.loc[holdout_start:].dropna(subset=["target"])

#     # Converting target to string for classification task
#     crypto_train["target"] = crypto_train["target"].astype(int).astype(str)
#     crypto_test["target"] = crypto_test["target"].astype(int).astype(str)
#     crypto_test["target"] = crypto_test["target"].astype(int).astype(str)
#     return crypto_train, crypto_test, crypto_holdout



#=============================================================
#====================BACKTESTING UTILITIES====================
#=============================================================

class TimeFrame():
    def __init__(self, data=None, index_col=None):
        """Initializing data"""
        if data is None:
            self.data = pd.DataFrame()
        else:
            self.data = data
            if index_col is None:
                pass
            else:
                self.data.set_index(index_col)
        if (self.data.index.dtype.__str__() in ['datetime64[ns]', 'datetime64']) or (self.data.empty):
            pass
        else:
            print("Requires one-dimensional datetime index")
            raise ValueError("index not datetime error")
    
    
    def check_datetime_index(self):
        """Check and enforce the main data index to a single-dimensional datetime"""
        if (self.data.index.dtype.__str__() in ['datetime64[ns]', 'datetime64']) or (self.data.empty):
            pass
        else:
            print("Requires one-dimensional datetime index")
            raise ValueError("index not datetime error")
            
            
    def add_data(self, data, index_col=None, values=None, lsuffix=None, rsuffix=None):
        """Joining another data source onto main data"""
        if not self.data.empty:
            if index_col is None:
                self.data = self.data.join(data, lsuffix=lsuffix, rsuffix=rsuffix)
            else:
                self.data = self.data.join(data.set_index(index_col), lsuffix=lsuffix, rsuffix=rsuffix)
        else:
            self.data = data
            if index_col is None:
                pass
            else:
                self.data.set_index(index_col)
        self.check_datetime_index()
    
    
    def load_data(self, fname, index_col=None):
        """Add data by loading a local csv"""
        data = pd.read_csv(fname)
        self.add_data(data, index_col=index_col)
    
    
    def multiply(self, col1, col2, out=None):
        """Multiply two columns together"""
        if out is None:
            out = col1 + "_times_" + col2
        self.data[out] = self.data[col1] * self.data[col2]
        self.check_datetime_index()
        
    
    def divide(self, col1, col2, out=None):
        """Dividing two columns"""
        if out is None:
            out = col1 + "_by_" + col2
        self.data[out] = self.data[col1] / self.data[col2].replace(0, np.nan)
        self.check_datetime_index()
    
    
    def aggregate(self, index_col=None, agg_dict=None):
        """Aggregate data at a specific level"""
        if index_col is None:
            groupby_col = self.data.index
        else:
            groupby_col = self.data[index_col]
        
        numeric_cols = self.data.dtypes[self.data.dtypes == float].index.tolist()
        if agg_dict is None:
            self.data = self.data.groupby(groupby_col)[[numeric_cols]].sum()
        else:
            self.data = self.data.groupby(groupby_col).agg(agg_dict)
        self.check_datetime_index()


class StockStrategy(TimeFrame):
    def __init__(self, data, price_col):
        super().__init__(data)
        self.price_col = price_col
        
        # Initializing with a null strategy
        self.signals = pd.DataFrame(index=data.index).sort_index()
        self.signals["signal"] = 0.0   
    
    
    #---------------------GENERATING STRATEGIES-------------------------
    #-------------------------------------------------------------------
    def generic_query_strategy(self, query=None, exit_query=None, plot=False):
        """Creating a generic strategy based upon input query.
           Running this method will erase the current strategy attribute
           and build a replacement"""
        if query is None:
            if hasattr(self, "strategy_query"):
                query = self.strategy_query
            else:
                raise
        
        if exit_query is None:
            if hasattr(self, "exit_query"):
                exit_query = self.exit_query
            else:
                pass
        else:
            pass
        
        self.signals = pd.DataFrame(index=self.data.index).sort_index()
        self.signals["signal"] = 0.0
        
        if exit_query is not None:
            buy_ind = self.data.query(query).index
            sell_ind = self.data.query(exit_query).index
            buy_sell = pd.Series(self.data.index.isin(buy_ind), index=self.data.index).map({True:1, False:np.nan})
            sell_times = pd.Series(self.data.index.isin(sell_ind), index=self.data.index).map({True:-1, False:np.nan}).dropna()
            buy_sell = pd.concat([sell_times, buy_sell])
            buy_sell = buy_sell[~buy_sell.index.duplicated(keep="first")]
            buy_sell = buy_sell.sort_index()

            curr_signal = 0
            signal_ind = []
            
            #
            # TODO: Vectorized Implementation
            #
            
            for x in buy_sell.iteritems():
                if x[1] == 1:
                    curr_signal = 1
                elif x[1] == -1:
                    curr_signal = 0
                else:
                    pass
                signal_ind.append(curr_signal)

            hold_signal = pd.Series(signal_ind, index=buy_sell.index)
            self.signals.loc[hold_signal.astype(bool)] = 1
        else:
            self.signals.loc[self.data.query(query).index] = 1
        self.signals = self.signals.sort_index()        
        self.signals["positions"] = self.signals["signal"].diff()
        self.strategy_query = query
        if exit_query is None:
            self.exit_query = "~({})".format(query)
        else:
            self.exit_query = exit_query
        
        if plot:
            fig, ax = plt.subplots()
            fig.set_size_inches(18.5, 10.5)
            self.data[self.price_col].sort_index().reset_index().astype({"Time":str}).set_index("Time").plot(ax=ax)
            ax2 = ax.twinx()
            self.signals["signal"].sort_index().reset_index().astype({"Time":str}).set_index("Time").plot(ax=ax2, color="pink")

    
    #---------------------BACKTESTING STRATEGIES-------------------------
    #--------------------------------------------------------------------
    def generate_portfolio(self, portfolio_df=None, price_col=None):
        """Creating basic portfolio attribute for object -- set of consecutive, historical prices and strategy
           positions as binary values -- 1/0 corresponding to hold/no-hold"""
        if price_col is not None:
            self.price_col = price_col
        
        if portfolio_df is None:
            portfolio_out_df = self.signals.copy()
            portfolio_out_df[self.price_col] = self.data[self.price_col]
            portfolio_out_df["base_return"] = portfolio_out_df[self.price_col].pct_change()
            self.portfolio = portfolio_out_df.astype({self.price_col:float, "base_return":float})
        else:
            portfolio_out_df = portfolio_df.copy()
            portfolio_out_df.index.names = ["Time"]
            portfolio_out_df.sort_index(inplace=True)
            if "positions" in portfolio_out_df:
                pass
            else:
                portfolio_out_df["positions"] = portfolio_out_df["signal"].diff()
            self.portfolio = portfolio_out_df.astype({self.price_col:float, "base_return":float})
    
    
    def track_portfolio(self, initial_capital, shares_per_position, price_col=None, portfolio=None):
        """Method to execute strategy over portfolio attribute. Once portfolio and strategy are in place
           this method will track the performance of the resulting strategy"""
        if portfolio is None:
            if hasattr(self, "portfolio"):
                portfolio = self.portfolio
            else:
                raise
        else:
            pass
        
        portfolio_out = portfolio.copy()
        if price_col is not None:
            self.price_col = price_col
        cash = initial_capital
        stock = 0
        stock_hist = [stock]
        cash_hist = [cash]
        currently_holding = False
        count = 0

        for row in portfolio.iterrows():
            test = row[1]
            hold = bool(test["signal"])
            if currently_holding:
                stock = stock * (1 + test["base_return"])                    
                if not hold:
                    if shares_per_position:
                        cash += stock
                        stock = 0
                    else:
                        cash = stock
                        stock = 0
                    currently_holding = False
            else:
                if hold:
                    if shares_per_position:
                        stock = shares_per_position*test[self.price_col]
                        if count > 0:
                            cash -= stock
                    else:
                        stock = cash
                        cash = 0
                    currently_holding = True
            cash_hist.append(cash)
            stock_hist.append(stock)
            count += 1
        
        portfolio_out["cash"], portfolio_out["stock"] = cash_hist[1:], stock_hist[1:]
        portfolio_out["total"] = portfolio_out["cash"] + portfolio_out["stock"]
        portfolio_out["returns"] = portfolio_out["total"].pct_change()
        return portfolio_out
    
    
    def backtest(self, initial_capital=10000, shares_per_position=100, transaction_fee=0.0, regenerate_portfolio=False, \
                 price_col=None):
        if hasattr(self, "portfolio"):
            if regenerate_portfolio:
                self.generate_portfolio(price_col=price_col)
            else:
                pass
        else:
            self.generate_portfolio(price_col=price_col)
        portfolio_out = self.track_portfolio(initial_capital, shares_per_position, price_col=price_col, portfolio=None)
        self.portfolio = portfolio_out
    
    
    def plot_backtest(self, date):
        fig = plt.figure()
        fig.set_size_inches(18.5, 10.5)
        ax1 = fig.add_subplot(111, ylabel='Portfolio value in $')
        ax2 = ax1.twinx()
        
        if date == "all":
            day_portfolio = self.portfolio.sort_index().reset_index().astype({"Time":str}).set_index("Time")
            day_signals = self.signals.sort_index().reset_index().astype({"Time":str}).set_index("Time")
        else:
            day_portfolio = self.portfolio.loc[self.portfolio.index.date == pd.Timestamp(date)]
            day_signals = self.signals.loc[self.signals.index.date == pd.Timestamp(date)]

        # Plot the equity curve in dollars
        day_portfolio['total'].plot(ax=ax1, lw=2.0)
        day_portfolio[self.price_col].plot(ax=ax2, color="orange")

        # Plot the "buy" trades against the equity curve
        if date != "all":
            ax1.plot(day_portfolio.loc[day_signals.positions == 1.0].index, 
                     day_portfolio.total[day_signals.positions == 1.0],
                     '^', markersize=10, color='m')

            # Plot the "sell" trades against the equity curve
            ax1.plot(day_portfolio.loc[day_signals.positions == -1.0].index, 
                     day_portfolio.total[day_signals.positions == -1.0],
                     'v', markersize=10, color='k')
        plt.title("Portfolio Value for {}".format(date))

        # Show the plot
        plt.show()
        
    #----------------------EVALUATING STRATEGIES-------------------------
    #--------------------------------------------------------------------
    def evaluate_strategy(self, metric):
        if metric == "sharpe":
            # Isolate the returns of your strategy
            returns = self.portfolio['returns']

            # annualized Sharpe ratio
            sharpe_ratio = np.sqrt(len(returns)) * (returns.mean() / returns.std())

            # Print the Sharpe ratio
            return sharpe_ratio
        elif metric == "max drawdown":
            # Define a trailing 252 trading day window
            window = 60
            
            # Calculate the max drawdown in the past window days for each day
            rolling_max = self.portfolio["total"].sort_index().rolling(window, min_periods=1).max()
            hourly_drawdown = self.portfolio["total"]/rolling_max - 1.0

            # Calculate the minimum (negative) daily drawdown
            max_hourly_drawdown = hourly_drawdown.rolling(window, min_periods=1).min()

            # Plot the results
            hourly_drawdown.plot()
            max_hourly_drawdown.plot()

            # Show the plot
            plt.show()
        elif metric == "cdgr":
            # Get the number of days in `aapl`
            mins = pd.Timedelta(self.portfolio.index[-1] - self.portfolio.index[0]).seconds/60

            # Calculate the CAGR
            cdgr = ((((self.portfolio[self.price_col][-1]) / self.portfolio[self.price_col][1])) ** (3600/mins)) - 1

            # Print CAGR
            return cdgr
        
        elif metric == "compound_return":
            returns =  (self.portfolio.returns + 1).cumprod() - 1
            baseline_returns = (self.portfolio[self.price_col].pct_change() + 1).cumprod() - 1
            baseline_final = baseline_returns[-1]
            portfolio_final = returns[-1]
            print("Baseline Asset Returns: {}%".format(round(baseline_returns[-1] * 100, 1)))
            print("Portfolio Returns: {}%".format(round(returns[-1] * 100, 1)))
            return returns[-1]
        
    
    #-------------------SAVING & LOADING STRATEGIES----------------------
    #--------------------------------------------------------------------
    def save_strategy(self, filename="strategy.yaml"):
        f = open(filename, "w+")
        f.write("---\n")
        f.write("enter query: {}\n".format(self.strategy_query))
        f.write("exit query: {}\n".format(self.exit_query))
        f.close()
    
    
    def load_strategy(self, filename="strategy.yaml"):
        # Reading in strategy queries
        with open(filename, "r") as stream:
            try:
                strategy = yaml.safe_load(stream)
            except yaml.YAMLError as exc:
                print(exc)
        query = strategy["enter query"]
        exit_query = strategy["exit query"]
        self.strategy_query = query
        if exit_query is None:
            self.exit_query = "~({})".format(query)
        else:
            self.exit_query = exit_query
    
    
    #---------------------------PLOTTING------------------------------
    #-----------------------------------------------------------------
    def plot_prices(self, date):
        fig = plt.figure()
        fig.set_size_inches(18.5, 10.5)
        ax1 = fig.add_subplot(111, ylabel='Portfolio value in $')
        date_df = self.data.loc[self.data.index.date == pd.Timestamp(date)]
        date_df[self.price_col].plot(ax=ax1)
        plt.show()

# Loading data

In [4]:
# Reading in data
crypto_df = load_crypto(account="kga72450.us-east-1",
                            warehouse="COMPUTE_WH",
                            database="CRYPTO",
                            schema="PUBLIC",
                            start_date_str="2022-04-01",
                            end_date_str=None,
                            snowflake_yaml="snowflake_key.yaml",
                            )

# Pivoting long to wide
crypto_wide, price_cols = preprocess1(crypto_df)
crypto_wide = crypto_wide.replace([np.inf, -np.inf], np.nan)

# Running Models

In [8]:
price_cols = ['AVAXUSDT', 'BATUSDT', 'BTCUSDT', 'CHZUSDT', 'DOTUSDT', 'ENJUSDT', 'ETHUSDT', 'FTMUSDT', 'GALAUSDT', 'LINKUSDT', 'LRCUSDT', 'MANAUSDT', 'MATICUSDT', 'SOLUSDT']
num_weeks_train, num_weeks_test, num_weeks_holdout = 4, 1, 1
train_start, test_start, holdout_start = get_split_dates(crypto_wide, num_weeks_train, num_weeks_test, num_weeks_holdout)

cross_corr_mat = get_cross_corr_matrix(crypto_wide.loc[train_start:test_start], price_cols, num_est=10)

coin_dict = {}

exception_dict = {}
for coin_of_interest in set(price_cols):
    crypto_train, crypto_test_holdout, crypto_return_clusters_train, crypto_return_clusters_test_holdout, \
    cluster_model_dict, outlier_model_dict = prepare_data(crypto_wide, coin_of_interest, [col for col in crypto_wide if ("BTCUSDT" in col) or ("ETHUSDT" in col) or (coin_of_interest in col)], train_start=train_start, test_start=test_start, holdout_start=holdout_start, cluster_num=None, 
                      cross_correlation_matrix=cross_corr_mat, cross_corr_thresh=0.50, outlier_column=True)
    holdout_strategy = crypto_test_holdout.sort_index().loc[holdout_start:]
    coin_dict[coin_of_interest] = {"cluster_models":cluster_model_dict, "outlier_models":outlier_model_dict, "holdout":holdout_strategy}
    return_periods = [5, 10, 60, 180]
    test_args = {"fold_strategy":"timeseries", "fold":3,}
    compare_args = {"include":["lr", "lda", "nb", "ada", "lightgbm", "ridge",]}
    exception_dict[coin_of_interest] = {}
    for return_period in return_periods:
        crypto_train_tmp, crypto_test_holdout_tmp, cluster_desc = add_target(coin_of_interest, crypto_train, crypto_test_holdout, return_period, test_start, crypto_return_clusters_train, crypto_return_clusters_test_holdout)
        crypto_train_tmp, crypto_test_tmp, crypto_holdout_tmp = split_df(crypto_train_tmp, crypto_test_holdout_tmp, holdout_start)
        try:
            best_models, best_model_finalized, logs = pycaret_automl(crypto_train_tmp, crypto_test_tmp, coin_of_interest, return_period, plot=False, download=False, save_folder="./crypto_models_1/{}/{}".format(coin_of_interest, return_period),
                                                                                              pycaret_setup_args=test_args,
                                                                                              pycaret_compare_args=compare_args)
        except Exception as e:
            traceback.print_exc()
            exception_dict[coin_of_interest][return_period] = Exception

In [10]:
holdout_start

Timestamp('2022-06-19 00:00:00')

### Uploading to GCS

In [ ]:
from google.cloud import storage
import glob

storage_client = storage.Client(project='crypto-341122')
bucket = storage_client.bucket("crypto-models-1")

for local_file in tqdm(glob.glob('./crypto_models/*/*/*'), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}'):
    blob = bucket.blob(local_file)
    blob.upload_from_filename(local_file)

# Crypto Data Utils

In [3]:
convert_time = lambda x: datetime.fromtimestamp(float(x)/1000)

def crypto_filter_times(crypto_df, date_str):
    return crypto_df.loc[crypto_df.OPEN_TIME.apply(convert_time) >= date_str]

In [11]:
crypto_test = crypto_filter_times(crypto_df, holdout_start.strftime("%Y-%m-%d"))
coin_dict = load_all_format_components_gcs(project="crypto-341122", bucket_name="formatter",)
crypto_formatted = data_formatter("ETHUSDT", crypto_test, coin_dict)

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [17]:
import tensorflow as tf

2022-07-09 14:18:37.457206: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-09 14:18:37.457264: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
crypto_formatted

,AVAXUSDT,BATUSDT,BTCUSDT,CHZUSDT,DOTUSDT,ENJUSDT,ETHUSDT,FTMUSDT,GALAUSDT,LINKUSDT,...,ETHUSDT_pct_change_5_clust,ETHUSDT_pct_change_60_clust,AVAXUSDT_OUTLIER,DOTUSDT_OUTLIER,ETHUSDT_OUTLIER,GALAUSDT_OUTLIER,ETHUSDT_PRED_5,ETHUSDT_PRED_10,ETHUSDT_PRED_60,ETHUSDT_PRED_180
Time,,,,,,,,,,,,,,,,,,,,,
2022-06-19 03:00:00,14.21,0.3005,18380.10,0.0841,6.90,0.401,950.70,0.2149,0.05177,5.92,...,2.0,0.0,-1.0,-1.0,-1.0,-1.0,1,1,4,3
2022-06-19 03:01:00,14.22,0.3006,18393.07,0.0842,6.90,0.401,951.60,0.2150,0.05181,5.94,...,2.0,0.0,-1.0,-1.0,-1.0,-1.0,1,1,4,3
2022-06-19 03:02:00,14.23,0.3012,18421.69,0.0844,6.91,0.401,953.15,0.2150,0.05187,5.94,...,2.0,0.0,-1.0,-1.0,-1.0,-1.0,2,2,3,3
2022-06-19 03:03:00,14.24,0.3012,18435.77,0.0844,6.90,0.401,954.04,0.2150,0.05188,5.94,...,0.0,0.0,-1.0,-1.0,-1.0,-1.0,2,2,3,3
2022-06-19 03:04:00,14.28,0.3023,18471.54,0.0845,6.92,0.402,956.01,0.2157,0.05201,5.99,...,1.0,0.0,-1.0,-1.0,-1.0,-1.0,2,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-25 23:56:00,21.35,0.4309,21516.95,0.1051,8.14,0.587,1244.11,0.3023,0.06911,7.28,...,1.0,3.0,1.0,-1.0,-1.0,-1.0,2,2,4,3
2022-06-25 23:57:00,21.39,0.4309,21513.58,0.1051,8.14,0.587,1243.94,0.3025,0.06897,7.28,...,1.0,3.0,-1.0,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN
2022-06-25 23:58:00,21.41,0.4315,21532.58,0.1052,8.16,0.589,1245.08,0.3051,0.06918,7.28,...,0.0,3.0,-1.0,-1.0,-1.0,-1.0,2,2,4,3


In [16]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 13.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 20.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.

# Strategy Selection

In [ ]:
# Formatting
coin_of_interest = "BTCUSDT"
strategy = StockStrategy(crypto_formatted.loc["2022-06-01":], coin_of_interest)

In [ ]:
# strategy.generic_query_strategy("BTCUSDT >= BTCUSDT_UP_2 + 200", exit_query="BTCUSDT <= BTCUSDT_LOW_2", plot=True)
strategy.generic_query_strategy("BTCUSDT >= BTCUSDT_UP_2 + 200", plot=True)

In [ ]:
# Purely prediction-based strategy
strategy.backtest(initial_capital=100, shares_per_position=None, transaction_fee=0.0)
strategy.plot_backtest("all")